In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column
from bokeh.io import output_notebook, show, save
from bokeh.models import CategoricalColorMapper
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Range1d, FactorRange
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6, Spectral10, Spectral4
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy import cumsum
from math import pi
from bokeh.palettes import Category20c
import plotly.graph_objects as go
import plotly.express as px
import warnings
from bokeh.models.glyphs import Text
from bokeh.transform import dodge
import plotly.express as px
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objs as go

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
path = ('../data/main.csv')

In [4]:
df_s2 = pd.read_csv(path)

In [5]:
df_s2 

,Morning Alertness,se_pcen,spt_pcen,sleepoffset_hr_pcen,L5VALUE,L5TIME_num,M10VALUE_daybefore,M10TIME_num_daybefore,meal_type_breakfast,meal_log_iauc_breakfast,meal_offset_to_breakfast_hr,age,sex,bmi,zygosity,sunrise_hr,dst,weekend,family_id,username
0,47.250000,0.001543,0.926686,0.955952,2.721177,0.668056,48.365305,7.834722,MCB,0.777249,-0.209444,53.645448,F,25.530409,MZ,7.800000,False,False,predict1009,predict10091
1,51.750000,0.034187,0.089186,-0.327381,2.175187,2.668056,17.615711,9.334722,OGTT,1.012362,1.586944,53.645448,F,25.530409,MZ,7.733333,False,False,predict1009,predict10091
2,59.000000,-0.023392,1.571131,1.355952,2.304887,4.668056,56.711167,10.001389,High Fibre,0.774941,0.625833,53.645448,F,25.530409,MZ,7.583333,False,True,predict1009,predict10091
3,50.000000,0.003077,0.091072,-0.458333,5.013151,0.334722,54.159403,7.168056,MCB,0.864187,2.024722,53.645448,F,24.272560,MZ,7.800000,False,False,predict1009,predict10092
4,53.800000,0.036777,-0.175595,0.058333,4.030731,2.668056,48.373843,10.168056,High Carb,1.248574,2.816667,53.645448,F,24.272560,MZ,7.783333,False,True,predict1009,predict10092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739,67.333333,0.022414,-1.448016,-1.704762,3.529514,1.834722,38.509619,9.334722,High Carb,1.688164,0.947222,55.822040,F,26.113602,MZ,7.000000,True,False,predict999,predict9991
6740,69.666667,0.017670,1.029762,0.778571,3.794901,5.168056,47.939220,12.501389,UK Average,1.493164,0.111111,55.822040,F,26.113602,MZ,7.083333,True,False,predict999,predict9991
6741,66.000000,-0.028763,-0.141071,-0.371429,2.229535,3.501389,24.526739,10.334722,OGTT,1.896814,0.264167,55.822040,F,26.113602,MZ,7.116667,True,False,predict999,predict9991
6742,72.666667,-0.033837,-0.886905,0.311905,3.391674,2.168056,40.709453,11.001389,High Fibre,1.604433,-1.285833,55.822040,F,26.113602,MZ,7.133333,True,True,predict999,predict9991


# Excercise Data

In [6]:
df_exercise = df_s2 [['L5VALUE', 'M10VALUE_daybefore', 'Morning Alertness']]

In [7]:
df_exercise

,L5VALUE,M10VALUE_daybefore,Morning Alertness
0,2.721177,48.365305,47.250000
1,2.175187,17.615711,51.750000
2,2.304887,56.711167,59.000000
3,5.013151,54.159403,50.000000
4,4.030731,48.373843,53.800000
...,...,...,...
6739,3.529514,38.509619,67.333333
6740,3.794901,47.939220,69.666667
6741,2.229535,24.526739,66.000000
6742,3.391674,40.709453,72.666667


In [8]:
print(df_exercise['M10VALUE_daybefore'].max())
print(df_exercise['M10VALUE_daybefore'].min())
print(df_exercise['M10VALUE_daybefore'].mean())

582.622116
8.768075
42.48872139650059


In [9]:
counts = df_exercise['M10VALUE_daybefore'].value_counts()
percent_above_100 = (counts[counts.index > 100].sum() / counts.sum()) * 100

print(f"{percent_above_100:.2f}% of people have an M10 value above 100")

1.85% of people have an M10 value above 100


In [10]:
df_exercise = df_exercise[df_exercise['M10VALUE_daybefore'] <= 100]


In [11]:
print(df_exercise['L5VALUE'].max())
print(df_exercise['L5VALUE'].min())
print(df_exercise['L5VALUE'].mean())

37.117257
0.067866
4.30070406753286


In [12]:
counts = df_exercise['L5VALUE'].value_counts()
percent_above_30 = (counts[counts.index > 30].sum() / counts.sum()) * 100

print(f"{percent_above_30:.2f}% of people have an L5 value above 30")

0.02% of people have an L5 value above 30


In [13]:
df_exercise['Legend'] = 'Data Points'

fig = px.scatter(
    df_exercise,
    x='M10VALUE_daybefore',
    y='Morning Alertness',
    color='Legend', 
    title="M10VALUE (Day Before) vs Morning Alertness",
    labels={
        "M10VALUE_daybefore": "M10VALUE (Day Before)",
        "Morning Alertness": "Morning Alertness"
    },
    color_discrete_sequence=["#1f77b4"]  
)

fig.update_layout(
    title=dict(
        text="M10VALUE (Day Before) vs Morning Alertness",
        font=dict(size=20)  
    ),
    xaxis=dict(
        title=dict(text="M10VALUE (Day Before)", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    legend=dict(
        title=dict(text="Metric", font=dict(size=14)), 
        font=dict(size=12),  
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    # height=500,  # Plot height
    # width=800  # Plot width
)

fig.show()


/var/folders/45/w5glyv3j3kb3yvsmvk3h71k00000gn/T/ipykernel_6710/1581800615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exercise['Legend'] = 'Data Points'


In [24]:
df_exercise['Legend'] = 'Data Points'

fig = px.scatter(
    df_exercise,
    x='L5VALUE',
    y='Morning Alertness',
    color='Legend',  
    title="L5VALUE vs Morning Alertness",
    labels={
        "L5VALUE": "L5VALUE",
        "Morning Alertness": "Morning Alertness"
    },
    color_discrete_sequence=["#1f77b4"]  
)

fig.update_layout(
    title=dict(
        text="L5VALUE vs Morning Alertness",
        font=dict(size=20)  
    ),
    xaxis=dict(
        title=dict(text="L5VALUE", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12) 
    ),
    legend=dict(
        title=dict(text="Metric", font=dict(size=14)),  
        font=dict(size=12), 
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    # height=500,  
    # width=800  
)

# Show the plot
fig.show()



/var/folders/45/w5glyv3j3kb3yvsmvk3h71k00000gn/T/ipykernel_6036/1410357892.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:

# Round off M10 values to the nearest integers
df_exercise['M10'] = df_exercise['M10VALUE_daybefore'].round()

df_grouped = df_exercise.groupby('M10', as_index=False).agg({'Morning Alertness': 'mean'})


fig = px.line(
    df_grouped,
    x='M10',
    y='Morning Alertness',
    title="M10 vs Morning Alertness with Trendline",
    labels={"M10": "M10 (Rounded)", "Morning Alertness": "Morning Alertness"}
)


model = sm.OLS(df_grouped['Morning Alertness'], sm.add_constant(df_grouped['M10'])).fit()


fig.add_traces(
    go.Scatter(
        x=df_grouped['M10'],
        y=model.predict(),
        mode='lines',
        line=dict(color='red', width=3, dash='dash'),
        name='Trendline'
    )
)


fig.update_layout(
    title=dict(
        text="M10 vs Morning Alertness with Trendline",
        font=dict(size=20) 
    ),
    xaxis=dict(
        title=dict(text="M10 (Rounded)", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    legend=dict(
        title=dict(text="Metric", font=dict(size=14)),  
        font=dict(size=12), 
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
   height=600,  
    width=1200   
)

# Show the plot
fig.show()


# This trend shows higher the M10 value the day before the better alertness the person has
# M10 is associated with more excersize

/var/folders/45/w5glyv3j3kb3yvsmvk3h71k00000gn/T/ipykernel_6710/2219305974.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
# Round off L5VALUE values to the nearest integers
df_exercise['L5'] = df_exercise['L5VALUE'].round()

df_grouped = df_exercise.groupby('L5', as_index=False).agg({'Morning Alertness': 'mean'})

fig = px.line(
    df_grouped,
    x='L5',
    y='Morning Alertness',
    title="L5 vs Morning Alertness with Trendline",
    labels={"L5": "L5 (Rounded)", "Morning Alertness": "Morning Alertness"}
)

model = sm.OLS(df_grouped['Morning Alertness'], sm.add_constant(df_grouped['L5'])).fit()


fig.add_traces(
    go.Scatter(
        x=df_grouped['L5'],
        y=model.predict(),
        mode='lines',
        line=dict(color='red', width=3, dash='dash'),
        name='Trendline'
    )
)

fig.update_layout(
    title=dict(
        text="L5 vs Morning Alertness with Trendline",
        font=dict(size=20)  
    ),
    xaxis=dict(
        title=dict(text="L5 (Rounded)", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12) 
    ),
    legend=dict(
        title=dict(text="Metric", font=dict(size=14)),  
        font=dict(size=12),  
        orientation="h", 
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    height=600, 
    width=1200  
)

# Show the plot
fig.show()

# This trend shows higher the L5 value the day before the lower the alertness the person has
# L5 is associated with more movement at the 5 least active hours of the day which includes the sleeping time

/var/folders/45/w5glyv3j3kb3yvsmvk3h71k00000gn/T/ipykernel_6710/2812951099.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Food Analysis

In [24]:
data=pd.read_csv('../data/main.csv')

In [25]:
data = data.drop(['family_id', 'username'], axis = 1)
print('missing data?', np.any(data.isnull()))
print(data[data.isnull().any(axis=1)])

missing data? False
Empty DataFrame
Columns: [Morning Alertness, se_pcen, spt_pcen, sleepoffset_hr_pcen, L5VALUE, L5TIME_num, M10VALUE_daybefore, M10TIME_num_daybefore, meal_type_breakfast, meal_log_iauc_breakfast, meal_offset_to_breakfast_hr, age, sex, bmi, zygosity, sunrise_hr, dst, weekend]
Index: []


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6744 entries, 0 to 6743
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Morning Alertness            6744 non-null   float64
 1   se_pcen                      6744 non-null   float64
 2   spt_pcen                     6744 non-null   float64
 3   sleepoffset_hr_pcen          6744 non-null   float64
 4   L5VALUE                      6744 non-null   float64
 5   L5TIME_num                   6744 non-null   float64
 6   M10VALUE_daybefore           6744 non-null   float64
 7   M10TIME_num_daybefore        6744 non-null   float64
 8   meal_type_breakfast          6744 non-null   object 
 9   meal_log_iauc_breakfast      6744 non-null   float64
 10  meal_offset_to_breakfast_hr  6744 non-null   float64
 11  age                          6744 non-null   float64
 12  sex                          6744 non-null   object 
 13  bmi               

In [27]:
df_train = data

In [28]:
df_train.dropna(how='any', inplace=True)

In [29]:
# Calculate the median values for each food type
medians = df_train.groupby('meal_type_breakfast')['meal_log_iauc_breakfast'].median().sort_values(ascending=False)
fig = px.box(
    df_train,
    x='meal_type_breakfast',
    y='meal_log_iauc_breakfast',
    color='meal_type_breakfast',
    color_discrete_map={
        'rice': '#1f77b4',
        'oats': '#ff7f0e',
        'bread': '#2ca02c',
        'noodles': '#d62728'
    },
    category_orders={'meal_type_breakfast': list(medians.index)}
)

fig.update_layout(
    title=dict(
        text="Breakfast Type vs Glucose Response",
        font=dict(size=20) 
    ),
    xaxis=dict(
        title=dict(text="Breakfast Type", font=dict(size=16)),  
        tickfont=dict(size=12) 
    ),
    yaxis=dict(
        title=dict(text="Glucose Response of Body", font=dict(size=16)),  
        tickfont=dict(size=12),  
        range=[0, 2.5]  
    ),
    legend=dict(
        title=dict(text="Breakfast Type", font=dict(size=14)),  
        font=dict(size=12),  
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    height=600,  
    width=1200  
)

# Show the plot
fig.show()

In [30]:
import plotly.express as px

# Define the order of the breakfast types based on their median values
breakfast_order = df_train.groupby('meal_type_breakfast')['meal_offset_to_breakfast_hr'].median().sort_values(ascending=False).index

fig = px.box(
    df_train,
    x='meal_type_breakfast',
    y='meal_offset_to_breakfast_hr',
    color='meal_type_breakfast',
    color_discrete_map={
        'rice': '#1f77b4',
        'oats': '#ff7f0e',
        'bread': '#2ca02c',
        'noodles': '#d62728'
    },
    category_orders={'meal_type_breakfast': breakfast_order}
)

fig.update_layout(
    title=dict(
        text="Breakfast Type vs Time for Second Meal",
        font=dict(size=20)  
    ),
    xaxis=dict(
        title=dict(text="Breakfast Type", font=dict(size=16)), 
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Time for Second Meal after Breakfast (in hours)", font=dict(size=16)),  
        tickfont=dict(size=12),  
        range=[-5, 12]  
    ),
    legend=dict(
        title=dict(text="Breakfast Type", font=dict(size=14)),  
        font=dict(size=12),  
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    height=600,  
    width=1200  
)

# Show the plot
fig.show()

In [31]:

breakfast_alertness = df_train.groupby('meal_type_breakfast')['Morning Alertness'].median().reset_index()
breakfast_alertness = breakfast_alertness.sort_values(by='Morning Alertness', ascending=False)
fig = px.box(
    df_train,
    x='meal_type_breakfast',
    y='Morning Alertness',
    color='meal_type_breakfast',
    color_discrete_map={
        'rice': '#1f77b4',
        'oats': '#ff7f0e',
        'bread': '#2ca02c',
        'noodles': '#d62728'
    },
    category_orders={'meal_type_breakfast': breakfast_alertness['meal_type_breakfast']}
)
fig.update_layout(
    title=dict(
        text="Breakfast Type vs Morning Alertness",
        font=dict(size=20)  
    ),
    xaxis=dict(
        title=dict(text="Breakfast Type", font=dict(size=16)), 
        tickfont=dict(size=12) 
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12),  
        range=[0, 125] 
    ),
    legend=dict(
        title=dict(text="Breakfast Type", font=dict(size=14)),  
        font=dict(size=12),  
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    height=600,  
    width=1200  
)

# Show the plot
fig.show()

In [32]:
df_train.dropna(how='any')

,Morning Alertness,se_pcen,spt_pcen,sleepoffset_hr_pcen,L5VALUE,L5TIME_num,M10VALUE_daybefore,M10TIME_num_daybefore,meal_type_breakfast,meal_log_iauc_breakfast,meal_offset_to_breakfast_hr,age,sex,bmi,zygosity,sunrise_hr,dst,weekend
0,47.250000,0.001543,0.926686,0.955952,2.721177,0.668056,48.365305,7.834722,MCB,0.777249,-0.209444,53.645448,F,25.530409,MZ,7.800000,False,False
1,51.750000,0.034187,0.089186,-0.327381,2.175187,2.668056,17.615711,9.334722,OGTT,1.012362,1.586944,53.645448,F,25.530409,MZ,7.733333,False,False
2,59.000000,-0.023392,1.571131,1.355952,2.304887,4.668056,56.711167,10.001389,High Fibre,0.774941,0.625833,53.645448,F,25.530409,MZ,7.583333,False,True
3,50.000000,0.003077,0.091072,-0.458333,5.013151,0.334722,54.159403,7.168056,MCB,0.864187,2.024722,53.645448,F,24.272560,MZ,7.800000,False,False
4,53.800000,0.036777,-0.175595,0.058333,4.030731,2.668056,48.373843,10.168056,High Carb,1.248574,2.816667,53.645448,F,24.272560,MZ,7.783333,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739,67.333333,0.022414,-1.448016,-1.704762,3.529514,1.834722,38.509619,9.334722,High Carb,1.688164,0.947222,55.822040,F,26.113602,MZ,7.000000,True,False
6740,69.666667,0.017670,1.029762,0.778571,3.794901,5.168056,47.939220,12.501389,UK Average,1.493164,0.111111,55.822040,F,26.113602,MZ,7.083333,True,False
6741,66.000000,-0.028763,-0.141071,-0.371429,2.229535,3.501389,24.526739,10.334722,OGTT,1.896814,0.264167,55.822040,F,26.113602,MZ,7.116667,True,False
6742,72.666667,-0.033837,-0.886905,0.311905,3.391674,2.168056,40.709453,11.001389,High Fibre,1.604433,-1.285833,55.822040,F,26.113602,MZ,7.133333,True,True


In [35]:
bins = np.arange(0, 1.6, 0.1)

df_train['bin'] = pd.cut(df_train['meal_log_iauc_breakfast'], bins=bins, labels=[f'{i:.1f}-{i+0.1:.1f}' for i in bins[:-1]])

df_grouped = df_train.groupby('bin')['Morning Alertness'].mean().reset_index()

z = np.polyfit(np.arange(len(df_grouped)), df_grouped['Morning Alertness'], 3)
p = np.poly1d(z)
xp = np.linspace(0, len(df_grouped) - 1, 100)

fig = px.line(
    df_grouped,
    x=np.arange(len(df_grouped)) / 10,
    y='Morning Alertness',
    title="Glucose Response vs Morning Alertness",
    labels={"x": "Glucose response of body after Breakfast", "Morning Alertness": "Morning Alertness"}
)

fig.add_traces(
    go.Scatter(
        x=xp / 10,
        y=p(xp),
        mode='lines+markers',
        line=dict(color='red', width=3, dash='dash'),
        name='Trendline'
    )
)

fig.update_layout(
    title=dict(
        text="Glucose Response vs Morning Alertness",
        font=dict(size=20) 
    ),
    xaxis=dict(
        title=dict(text="Glucose response of body after Breakfast", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    legend=dict(
        title=dict(text="Metrics", font=dict(size=14)),  
        font=dict(size=12),  
        orientation="h", 
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    height=600,  
    width=1200  
)

# Show the plot
fig.show()

In [36]:
fig = px.scatter(
    df_train,
    x='meal_offset_to_breakfast_hr',
    y='Morning Alertness',
    title="Meal Offset to Breakfast vs Morning Alertness",
    labels={
        "meal_offset_to_breakfast_hr": "Meal Offset to Breakfast (hours)",
        "Morning Alertness": "Morning Alertness"
    }
)

fig.update_layout(
    title=dict(
        text="Meal Offset to Breakfast vs Morning Alertness",
        font=dict(size=20)  
    ),
    xaxis=dict(
        title=dict(text="Meal Offset to Breakfast (hours)", font=dict(size=16)),  
        tickfont=dict(size=12)  
    ),
    yaxis=dict(
        title=dict(text="Morning Alertness", font=dict(size=16)),  
        tickfont=dict(size=12),  
        range=[0, 150]  
    ),
    legend=dict(
        title=dict(text="Metric", font=dict(size=14)),  
        font=dict(size=12),  
        orientation="h",  
        x=0.5,
        xanchor="center",
        y=-0.2
    ),
    height=600,  
    width=1200  
)

# Show the plot
fig.show()